## Invoking an ML API

This notebook demonstrates how to invoke a deployed ML model (in this case, the Google Cloud Natural Language API)
from a batch or streaming pipeline

We will use Apache Beam.

### Install Beam

Restart the kernel after installing Beam

In [3]:
%pip install --upgrade --quiet apache-beam[gcp]

Note: you may need to restart the kernel to use updated packages.


## Try out Beam

In [1]:
!rm -rf output.txt* beam-temp*

In [4]:
import apache_beam as beam
from apache_beam.ml.gcp import naturallanguageml as nlp

def parse_nlp_result(response):
    """
Pulls required info from a response that looks like this:

sentences {
  text {
    content: "I love walking along the Seine."
  }
  sentiment {
    magnitude: 0.699999988079071
    score: 0.699999988079071
  }
}
entities {
  name: "Seine"
  type: LOCATION
  metadata {
    key: "mid"
    value: "/m/0f3vz"
  }
  metadata {
    key: "wikipedia_url"
    value: "https://en.wikipedia.org/wiki/Seine"
  }
  salience: 1.0
  mentions {
    text {
      content: "Seine"
      begin_offset: 25
    }
    type: PROPER
  }
}
document_sentiment {
  magnitude: 0.699999988079071
  score: 0.699999988079071
}
language: "en"
    """
    return [
        # response, # entire string
        response.sentences[0].text.content,
        response.language,
        response.document_sentiment.score
    ]


features = nlp.types.AnnotateTextRequest.Features(
    extract_entities=True,
    extract_document_sentiment=True,
    extract_syntax=False
)

p = beam.Pipeline()
(p 
 | beam.Create(['Have you been to Paris?', 'I love walking along the Seine.', "C'est terrible"])
 | beam.Map(lambda x : nlp.Document(x, type='PLAIN_TEXT'))
 | nlp.AnnotateText(features)
 | beam.Map(parse_nlp_result)
 | beam.io.WriteToText('output.txt')
)
result = p.run()
result.wait_until_finish()

'DONE'

In [5]:
!cat output.txt*

['Have you been to Paris?', 'en', 0.0]
['I love walking along the Seine.', 'en', 0.699999988079071]
["C'est terrible", 'fr', -0.8999999761581421]


Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License